In [ ]:
import glob
import math
import uproot
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["svg"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

plt.style.use(hep.style.ROOT)  # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

path1 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=3.0eV'
path2 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=4.0eV'
path3 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=6.1eV'
path4 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=8.2eV'
path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV'

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(path + "/**.dat"))

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10
frame_srt = frame.groupby("filename",sort=False)

for key, item in frame_srt:
    allnames.append(key)
    
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
frame_srt.describe()

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
####################################################################################
#PHI THETEA electron (alpha and beta in Philipp convention)
#NOTE: the following phi and ctheta are not unique: len=20000
phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
phi_rad = phi_full * np.pi/180.

ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
theta_rad = np.arccos(ctheta_full)
theta_full = theta_rad*180./np.pi

#NOTE: the following phi and ctheta are unique: dim=(200,100)
# it is already the middle point, no need of doing it again
phiM = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi = [-179.1, 179.1]
cosM = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta) = [1,-1] 

#derivation from previous definition
# phiM=(phi[1:] + phi[:-1])/2
# cosM=(ctheta[1:] + ctheta[:-1])/2

# In the 2-D case with inputs of length M and N, the outputs are of shape (N, M) for ‘xy’ indexing and (M, N) for ‘ij’ indexing. Cartesia "xy" by default
phiMM, cosMM = np.meshgrid(phiM, cosM)
phiMM_rad, thetaMM_rad = np.meshgrid(phiM*np.pi/180., np.arccos(cosM))
####################################################################################


####################################################################################
#PHI THETA photon in sperical coordinates in respect to the LF
#has to be adapted to the THEORY: 0.835,-0.835,-165,165
#accordinf to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
#accordinf to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

#optimized range according to philipp
phicos_PHOTON = np.around(np.array(list(itertools.product(np.linspace(-180,180,12).tolist(),np.cos(np.linspace(0,np.pi,6).tolist())))),3)
cosphi_PHOTON = np.around(np.array(list(itertools.product(np.cos(np.linspace(0,np.pi,6).tolist()),np.linspace(-180,180,12).tolist()))),3)

#maual spacing
# phicos_MFPAD = np.around(np.array(list(itertools.product(np.linspace(-165,165,12).tolist(),np.linspace(0.835,-0.835,6).tolist()))),3)
# cosphi_MFPAD = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

#plotly coordiantes arranged according to phi
#NOTE: x is always the 12 member array 
xgo_phi=[col[0] for col in phicos_PHOTON]
ygo_phi=[col[1] for col in phicos_PHOTON]

#plotly coordiantes arranged according to cos
#NOTE: x is always the 12 member array
xgo_cos=[col[1] for col in cosphi_PHOTON]
ygo_cos=[col[0] for col in cosphi_PHOTON]

print("pihM ", phiM.shape, "cosM ", cosM.shape)
print(type(phiM), type(cosM))
####################################################################################

#test initial plotting
counts_1_1 = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values
counts_1_1 = counts_1_1.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_1_1[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_1_1[1,:]), "type ", type(counts_1_1))

In [ ]:
# (np.array((xgo_cos[1:] + xgo_cos[:-1])/2))
np.array(xgo_cos)

In [ ]:
#manual definiton of light vector in MF. The order is the one matching the MFPADs of Philipp
#accordinf to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
#accordinf to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

#manual remaping of cosphi not adjusted
#cosphi_adj=list(fugi.remap(cosphi_th,0.835,-0.835,-165,165))
#cosphi_adj=list(fugi.remap(cosphi_th,-1.,1.,-180,180))
#cosphi_adj=np.around(cosphi_adj,3)

cosphi_PHOTON_temp = np.around(np.array(list(itertools.product(np.cos(np.linspace(0,np.pi,7).tolist()),np.linspace(-180,180,13).tolist()))),3)
cosphi_adj = fugi.avg_func(cosphi_PHOTON_temp)

print(cosphi_adj[0][0])
#look to the values in b1 map and found the MFPAD

dfind=pd.DataFrame(cosphi_adj, columns=["ctheta","phi"])
#the assignment of this colums depens on frame_str to be SORTED or UNSORTED
#the most straightforward (ad right) aproach is UNSORTED -> 1_1, 1_2 instead of 1_1, 1_10
#NOTE: first number = cos(theta), second number = PHI of photon rotation
dfind["item"]=allnames

#number of the MFPAD
cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values

#actual values of cos and phi
cosphi_adj_cos=np.array((dfind.sort_values(by=["ctheta","phi"]))[["ctheta","phi"]].values)
cosphi_adj_phi=np.array((dfind.sort_values(by=["phi","ctheta"]))[["ctheta","phi"]].values)

# find the photon rotation vakues of a specific set
dfind.sort_values(by=["phi", "ctheta"])
dfind.loc[dfind["item"] == "4_7"]

In [ ]:
# plt.imshow(z_matrix)
plt.imshow(counts_1_1.T,extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_title("MFPAD S enanant 1_1")
plt.colorbar()

In [ ]:
# countour plot
cmap = mpl.cm.viridis
cmap.set_under('w')

fig1, ax = plt.subplots(constrained_layout=True)

CS = ax.contourf(phiM,cosM,counts_1_1.T, levels=20, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_title("MFPAD S enanant 1_1")
fig1.colorbar(CS)

In [ ]:
# convert the counts into a 2D np array and sort according to cos and phi
#NOTE: theta is along the z_MF and phi is atan(x_MF,y_MF)
from numpy.core.defchararray import multiply

counts_temp=[] #standard cos sorted S enantiomer
MFPAD_S_phi=[]
MFPAD_S_cos=[]
MFPAD_R_phi=[]
MFPAD_R_cos=[]

#To have the right matching of MFPADs array and allnames frame_srt shouldn't be UNSORTED (see first cell)
#if UNSORTED MFPAD[0] = filename 1_1; MFPAD[1] = filename 1_2 
#if SORTED MFPAD[0] = filename 1_1; MFPAD[1] = filename 1_10 
for group in frame_srt:
    counts_temp.append(group[1]["value"].to_numpy().reshape(200,100))
    # counts_temp.append(group[0])

#sorting: MFPAD are already the type np.array    
counts_temp=np.array(counts_temp)
MFPAD_S_phi=fugi.sorting_array(counts_temp, cosphi_PHOTON, cosphi_th, 2) #sorted according to phi
MFPAD_S_cos=fugi.sorting_array(counts_temp, cosphi_PHOTON, cosphi_th, 1) #sorted according to cos

#projection respect to cos is the standard calculation
#integration of MFPAD onto cos or phi for S enantiomer IN MF
cos_projthS_phi=fugi.projection(MFPAD_S_phi,a=0)
phi_projthS_phi=fugi.projection(MFPAD_S_phi,a=1)
cos_projthS_cos=fugi.projection(MFPAD_S_cos,a=0)
phi_projthS_cos=fugi.projection(MFPAD_S_cos,a=1)

#flipping for R enantiomer and sorting
for j,el in enumerate(MFPAD_S_phi):
    tempcos_phi=np.flip(MFPAD_S_phi[j], axis=0)
    MFPAD_R_phi.append(np.flip(tempcos_phi, axis=1))

    tempcos_cos=np.flip(MFPAD_S_cos[j], axis=0)
    MFPAD_R_cos.append(np.flip(tempcos_cos, axis=1))

#converting counts into np.array
MFPAD_R_phi=np.array(MFPAD_R_phi)
MFPAD_R_cos=np.array(MFPAD_R_cos)

#integration of of MFPAD onto cos or phi for R enantiomer
cos_projthR_phi=fugi.projection(MFPAD_R_phi, a=0)
phi_projthR_phi=fugi.projection(MFPAD_R_phi, a=1)
cos_projthR_cos=fugi.projection(MFPAD_R_cos, a=0)
phi_projthR_cos=fugi.projection(MFPAD_R_cos, a=1)


print(np.array(MFPAD_S_phi).shape)
print(np.array(MFPAD_S_cos).shape)
print(np.array(MFPAD_R_phi).shape)
print(np.array(MFPAD_R_cos).shape)
print(np.array(cos_projthS_phi).shape)
print(np.array(cos_projthR_phi).shape)

In [ ]:
MFPAD_S_cos_norm=[]
MFPAD_S_phi_norm=[]
MFPAD_R_cos_norm=[]
MFPAD_R_phi_norm=[]
mPECD_cos=[]
mPECD_phi=[]

#///////////////////////// cos ///////////////////////////////////
for colsc,colrc in zip(MFPAD_S_cos,MFPAD_R_cos):
    MFPAD_S_cos_norm.append(fugi.normalise_matrix(colsc,normtype=1))
    MFPAD_R_cos_norm.append(fugi.normalise_matrix(colrc,normtype=1))

MFPAD_S_cos_norm=np.array(MFPAD_S_cos_norm)
MFPAD_R_cos_norm=np.array(MFPAD_R_cos_norm)
MFPAD_S_cos_norm_sum=np.sum(MFPAD_S_cos_norm,axis=0)
MFPAD_R_cos_norm_sum=np.sum(MFPAD_R_cos_norm,axis=0)

mPECD_cos=np.divide(np.subtract(MFPAD_R_cos_norm,MFPAD_S_cos_norm),np.add(MFPAD_R_cos_norm,MFPAD_S_cos_norm))
mPECD_cos_sum=np.divide(np.subtract(MFPAD_R_cos_norm_sum,MFPAD_S_cos_norm_sum),np.add(MFPAD_R_cos_norm_sum,MFPAD_S_cos_norm_sum))

#///////////////////////// phi ///////////////////////////////////
for colsp,colrp in zip(MFPAD_S_phi,MFPAD_R_phi):
    MFPAD_S_phi_norm.append(fugi.normalise_matrix(colsp,normtype=1))
    MFPAD_R_phi_norm.append(fugi.normalise_matrix(colrp,normtype=1))

MFPAD_S_phi_norm=np.array(MFPAD_S_phi_norm)
MFPAD_R_phi_norm=np.array(MFPAD_R_phi_norm)
MFPAD_S_phi_norm_sum=np.sum(MFPAD_S_phi_norm,axis=0)
MFPAD_R_phi_norm_sum=np.sum(MFPAD_R_phi_norm,axis=0)

#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

mPECD_phi=np.divide(np.subtract(MFPAD_R_phi_norm,MFPAD_S_phi_norm),np.add(MFPAD_R_phi_norm,MFPAD_S_phi_norm))
mPECD_phi_sum=np.divide(np.subtract(MFPAD_R_phi_norm_sum,MFPAD_S_phi_norm_sum),np.add(MFPAD_R_phi_norm_sum,MFPAD_S_phi_norm_sum))

#NOTE: R-S nad S-R seems to have no influence on b1 map
cPECD_cos =  np.divide(np.subtract(cos_projthR_cos,cos_projthS_cos),np.add(cos_projthR_cos,cos_projthS_cos))
cPECD_phi =  np.divide(np.subtract(cos_projthR_phi,cos_projthS_phi),np.add(cos_projthR_phi,cos_projthS_phi))

#it is somehow unconventional, but let's normalise the error with the square root of all counts
errPECD_cos = np.sqrt(1/(cos_projthR_cos+cos_projthS_cos))
errPECD_phi = np.sqrt(1/(cos_projthR_phi+cos_projthS_phi))

In [ ]:
MFPAD_R_phi_LF=[]
MFPAD_R_cos_LF=[]

#rotation of the MFPAD and collection of the cosx
MFPAD_S_phi_LF,cos_S_phi_LF,phi_S_phi_LF=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad,phi_rad,cosphi_adj_phi,phiMM,cosMM,method="linear")
# MFPAD_R_phi_LF,cos_R_phi_LF,phi_R_phi_LF=fugi.rot3d_MFPAD(MFPAD_R_phi,theta_rad,phi_rad,cosphi_adj_phi,phiMM,cosMM,method="linear")
MFPAD_S_cos_LF,cos_S_cos_LF,phi_S_cos_LF=fugi.rot3d_MFPAD(MFPAD_S_cos,theta_rad,phi_rad,cosphi_adj_cos,phiMM,cosMM,method="linear")
# MFPAD_R_cos_LF,cos_R_cos_LF,phi_R_cos_LF=fugi.rot3d_MFPAD(MFPAD_R_cos,theta_rad,phi_rad,cosphi_adj_cos,phiMM,cosMM,method="linear")

#remove nan
MFPAD_S_phi_LF=np.nan_to_num(MFPAD_S_phi_LF)
MFPAD_S_cos_LF=np.nan_to_num(MFPAD_S_cos_LF)

#flipping for R enantiomer and sorting
for el in MFPAD_S_phi_LF:
    tempcos_phi=np.flip(el, axis=0)
    MFPAD_R_phi_LF.append(np.flip(tempcos_phi, axis=1))

for el in MFPAD_S_cos_LF:
    tempcos_cos=np.flip(el, axis=0)
    MFPAD_R_cos_LF.append(np.flip(tempcos_cos, axis=1))

#integration of MFPAD onto cos or phi for S enantiomer
cosx_S_phi_LF=fugi.projection(MFPAD_S_phi_LF, a=0)
phi_projthS_phi_LF=fugi.projection(MFPAD_S_phi_LF, a=1)
cosx_S_cos_LF=fugi.projection(MFPAD_S_cos_LF, a=0)
phi_projthS_cos_LF=fugi.projection(MFPAD_S_cos_LF, a=1)

#integration of MFPAD onto cos or phi for R enantiomer
cosx_R_phi_LF=fugi.projection(MFPAD_R_phi_LF, a=0)
phi_projthR_phi_LF=fugi.projection(MFPAD_R_phi_LF, a=1)
cosx_R_cos_LF=fugi.projection(MFPAD_R_cos_LF, a=0)
phi_projthR_cos_LF=fugi.projection(MFPAD_R_cos_LF, a=1)


# #//////////////////////////// cos PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#NOTE: R-S and S-R seems to have no influence on b1 map
cPECD_LF_phi = np.nan_to_num(np.divide(np.subtract(cosx_R_phi_LF,cosx_S_phi_LF),np.add(cosx_R_phi_LF,cosx_S_phi_LF)))
cPECD_LF_cos = np.nan_to_num(np.divide(np.subtract(cosx_R_cos_LF,cosx_S_cos_LF),np.add(cosx_R_cos_LF,cosx_S_cos_LF)))

#it is somehow unconventional, but let's normalise the error with the square root of all counts
errPECD_LF_cos = np.nan_to_num(np.sqrt(1/(cosx_R_cos_LF+cosx_S_cos_LF)))
errPECD_LF_phi = np.nan_to_num(np.sqrt(1/(cosx_R_phi_LF+cosx_S_phi_LF)))


#////////////////////////////// NORM \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
MFPAD_S_cos_LF_norm=[]
MFPAD_S_phi_LF_norm=[]
MFPAD_R_cos_LF_norm=[]
MFPAD_R_phi_LF_norm=[]
mPECD_LF_cos=[]
mPECD_LF_phi=[]

#///////////////////////// cos ///////////////////////////////////
for colsc,colrc in zip(MFPAD_S_cos_LF,MFPAD_R_cos_LF):
    MFPAD_S_cos_LF_norm.append(fugi.normalise_matrix(colsc,normtype=1))
    MFPAD_R_cos_LF_norm.append(fugi.normalise_matrix(colrc,normtype=1))

MFPAD_S_cos_LF_norm=np.array(MFPAD_S_cos_LF_norm)
MFPAD_R_cos_LF_norm=np.array(MFPAD_R_cos_LF_norm)
MFPAD_S_cos_LF_norm_sum=np.sum(MFPAD_S_cos_LF_norm,axis=0)
MFPAD_R_cos_LF_norm_sum=np.sum(MFPAD_R_cos_LF_norm,axis=0)

MFPAD_S_cos_LF_sum=np.sum(MFPAD_S_cos_LF,axis=0)
MFPAD_R_cos_LF_sum=np.sum(MFPAD_R_cos_LF,axis=0)

#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#NOTE: it makes no sense because the rotated version doesn't have anything different from the roated at the moment. cos and phi have to be introduced.
mPECD_LF_cos=np.divide(np.subtract(MFPAD_R_cos_LF_norm,MFPAD_S_cos_LF_norm),np.add(MFPAD_R_cos_LF_norm,MFPAD_S_cos_LF_norm))
mPECD_LF_cos_sum=np.divide(np.subtract(MFPAD_R_cos_LF_norm_sum,MFPAD_S_cos_LF_norm_sum),np.add(MFPAD_R_cos_LF_norm_sum,MFPAD_S_cos_LF_norm_sum))

#///////////////////////// phi ///////////////////////////////////
for colsp,colrp in zip(MFPAD_S_phi_LF,MFPAD_R_phi_LF):
    MFPAD_S_phi_LF_norm.append(fugi.normalise_matrix(colsp,normtype=1))
    MFPAD_R_phi_LF_norm.append(fugi.normalise_matrix(colrp,normtype=1))

MFPAD_S_phi_LF_norm=np.array(MFPAD_S_phi_LF_norm)
MFPAD_R_phi_LF_norm=np.array(MFPAD_R_phi_LF_norm)
MFPAD_S_phi_LF_norm_sum=np.sum(MFPAD_S_phi_LF_norm,axis=0)
MFPAD_R_phi_LF_norm_sum=np.sum(MFPAD_R_phi_LF_norm,axis=0)

MFPAD_S_phi_LF_sum=np.sum(MFPAD_S_phi_LF,axis=0)
MFPAD_R_phi_LF_sum=np.sum(MFPAD_R_phi_LF,axis=0)

#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#NOTE: it makes no sense because the rotated version doesn't have anything different from the roated at the moment. cos and phi have to be introduced.
mPECD_LF_phi=np.divide(np.subtract(MFPAD_R_phi_LF_norm,MFPAD_S_phi_LF_norm),np.add(MFPAD_R_phi_LF_norm,MFPAD_S_phi_LF_norm))
mPECD_LF_phi_sum=np.divide(np.subtract(MFPAD_R_phi_LF_norm_sum,MFPAD_S_phi_LF_norm_sum),np.add(MFPAD_R_phi_LF_norm_sum,MFPAD_S_phi_LF_norm_sum))

plt.plot(phi_full.reshape(-1),".")
plt.plot(phi_S_phi_LF[4].reshape(-1),"o")

plt.plot(ctheta_full.reshape(-1),".")
plt.plot(cos_S_phi_LF[4].reshape(-1),"o")

fig1, ax = plt.subplots(1,2, figsize=(10,5), constrained_layout=True)
ax[0].scatter(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1),marker=".",s=1)
ax[1].scatter(phiMM,cosMM,marker=".",s=1)

#too intesive on the CPU
from scipy import interpolate

i2d = interpolate.interp2d(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1),MFPAD_S_phi_LF[4].reshape(-1))
znew = i2d(phiM.reshape(-1), cosM.reshape(-1))

import matplotlib.tri as tri 
triang = tri.Triangulation(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1))
interpolator = tri.CubicTriInterpolator(triang, MFPAD_S_phi_LF[4].reshape(-1), kind='geom')
phiMM_test, cosMM_test = np.meshgrid(phiM, cosM)
zinterp = interpolator(phiMM_test,cosMM_test)
#zinterp = tri.interp_cubic_geom(phiM,cosM)

#even with 4 subdivided triangles
#refiner = tri.UniformTriRefiner(triang)
#tri_refi, z_test_refi = refiner.refine_field(MFPAD_S_phi_LF[4].reshape(-1), subdiv=1)

#ax[0,1].contourf(phiMM,cosMM,zinterp)
#ax[0,1].set_xlabel('phi [DEG]')
#ax[0,1].set_ylabel('cos(theta) [adm]')
#ax[0,1].set_title("interp2d rot")


#equivalente to interpolator methoh
#to be used after the 3D rotation of the MFPAD passing through cartesian coordiantes
from scipy.interpolate import griddata
zi = griddata((phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1)), MFPAD_S_phi_LF[4].reshape(-1), (phiMM, cosMM), method='linear')
zi_zero = np.nan_to_num(zi,np.nan)


In [ ]:
#TEST: rotation in 2d without reporting the 
fugi = reload(fugi)
tMFPAD_S_phi_LF,tcos_S_phi_LF,tphi_S_phi_LF=fugi.rot2d_MFPAD(MFPAD_S_phi,ctheta_full,phi_full,cosphi_adj_phi,phiMM,cosMM,method="linear")
tMFPAD_S_phi_LF=(np.nan_to_num(tMFPAD_S_phi_LF,np.nan)).reshape(72,200,100)

In [ ]:
#NOTE: after the triangluation one has to use phiMM and cosMM
fig1, ax = plt.subplots(1,3, figsize=(12,5), constrained_layout=True)
# fig1, ax = plt.subplots(constrained_layout=True)

# ax[0].contourf(triang,zinterp.reshape(-1))
# ax[0].contourf(phiMM,cosMM,MFPAD_S_phi_LF[4].T)
ax[0].contourf(phiMM,cosMM,MFPAD_S_phi[0].T)
# ax[0].contourf(phiMM,cosMM,MFPAD_S_phi_LF_norm_sum.T)
# ax[0].contourf(phiMM, cosMM, MFPAD_temp)
# ax[0].contourf(znew.T)
ax[0].set_xlabel('phi [DEG]')
ax[0].set_ylabel('cos(theta) [adm]')
ax[0].set_title("original")

ax[1].contourf(phiMM, cosMM, tMFPAD_S_phi_LF[0].T)
# ax[1].tricontourf(phi_full.reshape(-1), ctheta_full.reshape(-1), tMFPAD_S_phi_LF[4].reshape(-1))
# ax[1].tricontourf(tphi_S_phi_LF[4].reshape(-1), tcos_S_phi_LF[4].reshape(-1), tMFPAD_S_phi_LF[4].reshape(-1))
# ax[1].contourf(phiMM, cosMM, test[4].T)
# ax[1].tricontourf(tphi_S_phi_LF[4].reshape(-1), tcos_S_phi_LF[4].reshape(-1), tMFPAD_S_phi_LF[4].reshape(-1))
# ax[1].contourf(tphi_S_phi_LF[4], tcos_S_phi_LF[4], tMFPAD_S_phi_LF[4])
# ax[1].contourf(phi_S_phi_LF[4], cos_S_phi_LF[4], MFPAD_S_phi_LF[4])
# ax[1].contourf(phiM, cosM, tMFPAD_S_phi_LF[4].reshape(200,100).T)
ax[1].set_xlabel('phi [DEG]')
ax[1].set_ylabel('cos(theta) [adm]')
ax[1].set_title("rot2D in sperical")

ax[2].contourf(phiMM,cosMM,MFPAD_S_phi_LF[0].T)
ax[2].set_xlabel('phi [DEG]')
ax[2].set_ylabel('cos(theta) [adm]')
ax[2].set_title("rot3D with cartesian")


In [ ]:
# all the 72 plots S ENANTIOMER MOLECULAR FRAME
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    # ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phiMM,cosMM,MFPAD_S_phi[i].T)
    # ax.text(0, 0, "S_en "+str(cosphi_th_phi[i]), ha="center", va="center", color="red",fontsize=18)
    ax.text(0, 0, str(cosphi_adj_phi[i].T), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.tight_layout()
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_phi.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots S ENANTIOMER MOLECULAR FRAME
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    # ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phiMM,cosMM,MFPAD_S_cos[i].T)
    # ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)
    ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.tight_layout()
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_cos.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots S ENANTIOMER MOLECULAR FRAME without x and y axis
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),MFPAD_S_phi_LF[i].reshape(-1))
    # ax.contourf(phiMM,cosMM,MFPAD_S_phi_LF[i].T)
    ax.contourf(MFPAD_S_phi_LF[i].T)
    ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig2a)
fig2a.tight_layout()
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_phi_LF.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots S ENANTIOMER LABORATORY FRAME 2D ROTATED
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    ax.contourf(phiMM,cosMM,MFPAD_S_phi_LF[i].T)
    ax.text(0, 0, "S_en "+str(cosphi_th_phi[i]), ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig2a)
fig2a.tight_layout()
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_phi_LF_with axes.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots S ENANTIOMER LABORATORY FRAME 2D ROTATED
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    ax.contourf(phiMM,cosMM,MFPAD_S_cos_LF[i].T)
    ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig2a)
fig2a.tight_layout()
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_cos_LF_with axes.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots: no borders & sharing axis
#S ENANTIOMER

# fig3, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row') #figsize in inche by defauls: 1 in = 2.8 cm
fig3, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row') #downsized
fig3.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)

fugi.overlaygraph(fig3)
fig3.tight_layout()
plt.show()

In [ ]:
# all the 72 plots FOR R ENANTIOMER NOT SORTED: to be compare directly with experimental data
#fig4, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig4, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig4.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_R_cos[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.text(0, 0, "R_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig4)
fig4.tight_layout()
plt.show()

In [ ]:
# all the 72 plots FOR R ENANTIOMER NOT SORTED: to be compare directly with experimental data
# fig4a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig4a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig4a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_R_phi[i].reshape(200,100), levels=20, vmin=0.0001)

fugi.overlaygraph(fig4a)
fig4a.tight_layout()
plt.show()

In [ ]:
#TEST of the PECD because of manual inversion to R enantiomer in MF
print(np.sum(cos_projthR_cos[60]))
plt.plot(errPECD_cos[60])
plt.plot(cPECD_cos[60])
plt.plot(cos_projthR_cos[60])
plt.plot(cos_projthS_cos[60])

In [ ]:
fig1, ax = plt.subplots(1,3, constrained_layout=True)
#along z axis
ax[0].plot(cos_projthR_cos[0])
ax[0].plot(cos_projthS_cos[0])

#along x axis
ax[1].plot(cosx_R_cos_LF[0])
ax[1].plot(cosx_S_cos_LF[0])

#along y axis
ax[2].plot(phi_projthR_cos_LF[0])
ax[2].plot(phi_projthS_cos_LF[0])

In [ ]:
#TEST of the PECD because of manual inversion to R enantiomer in LF
print(np.sum(cosx_R_cos_LF[60]))
plt.plot(errPECD_LF_cos[60])
plt.plot(cPECD_LF_cos[60])
plt.plot(cosx_R_cos_LF[60])
plt.plot(cosx_S_cos_LF[60])

In [ ]:
# interpolation modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.err = np.asarray(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        #chi2 function
        return np.sum((self.y - ym)**2/(self.err**2))

        #custom fucntion
        # z = (self.y - ym) / self.err**2
        # return np.sum(z**2)


# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30*x**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

In [ ]:
#SELECT HERE THE INPUT

from scipy.interpolate import make_interp_spline, BSpline

#NOTE: x_data has to be monotonicalli INCREASING!
x_data = np.flip(cosM)
y_data = cPECD_LF_cos[7]
y_err = errPECD_LF_cos[7]

lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

m = Minuit(lsq,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0) # initial values are 0 by default
m.limits = limits
m.errordef = Minuit.LEAST_SQUARES
m.migrad() # run optimiser
m.hesse() # run covariance estimator

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

param_matrix_cos = [[0 for i in range(6)] for j in range(72)] #i col, j row

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
# custom_ylim = (-0.2,0.2)
custom_ylim = (-1, 1)
limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
# limits=[(-1, 1), (-1, 1), (None, None), (None, None), (None, None), (None, None)]

for i,ax in zip(range(72),axes.flatten()): #72 fixed because of the photon directions, equal to range(len(cosphi))

    # #After rotation in LF
    #NOTE: x_data has to be monotonicalli INCREASING!
    x_data = np.flip(cosM)
    y_data = cPECD_LF_cos[i]
    y_err = errPECD_LF_cos[i]

    #original in MF
    #why flipped? x input has to be monotonic increasing array
    # x_data = np.flip(cosM)
    # y_data = cPECD_cos[i]
    # y_err = errPECD_cos[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0) # initial values are 0 by default
    m.limits = limits
    m.errordef = Minuit.LEAST_SQUARES
    m.migrad() # run optimiser
    # m.hesse() # run covariance estimator
    # with limints intead of m.migrad()
    # m.scan(ncall=10)
    
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values), k=5)    
    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)

    ax.plot(x_data_new,bspl(x_data_new))
    ax.scatter(x_data,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    # ax.errorbar(x_data,y_data, yerr=y_err, fmt="o")

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_cos[i][counter]=((m.values[el], m.errors[el]))

#numpy tranformation!!!
param_matrix_cos=np.array(param_matrix_cos)

fugi.overlaygraph(fig)
fig.tight_layout()
plt.show()

#test for the validity of all the parameters
print(param_matrix_cos.shape)
print(param_matrix_cos[:,0,0].shape)
print(param_matrix_cos)

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('b1 map Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[:,0,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[:,0,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
param_matrix_cos.shape

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(6):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

param_matrix_phi = [[0 for i in range(6)] for j in range(72)] #i col, j row

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
# custom_ylim = (-0.2,0.2)
custom_ylim = (-1, 1)
limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
# limits=[(-1, 1), (-1, 1), (None, None), (None, None), (None, None), (None, None)]

for i,ax in zip(range(72),axes.flatten()): #72 fixed because of the photon directions, equal to range(len(cosphi))

    # #After rotation in LF
    #NOTE: x_data has to be monotonicalli INCREASING!
    x_data = np.flip(cosM)
    y_data = cPECD_LF_phi[i]
    y_err = errPECD_LF_phi[i]

    #original in MF
    #why flipped? x input has to be monotonic increasing array
    # x_data = np.flip(cosM)
    # y_data = cPECD_cos[i]
    # y_err = errPECD_cos[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0) # initial values are 0 by default
    m.limits = limits
    m.errordef = Minuit.LEAST_SQUARES
    m.migrad() # run optimiser
    # m.hesse() # run covariance estimator
    # with limints intead of m.migrad()
    # m.scan(ncall=10)
    
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values), k=5)    
    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)

    ax.plot(x_data_new,bspl(x_data_new))
    ax.scatter(x_data,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    # ax.errorbar(x_data,y_data, yerr=y_err, fmt="o")

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_phi[i][counter]=((m.values[el], m.errors[el]))

#numpy tranformation!!!
param_matrix_phi=np.array(param_matrix_phi)

fugi.overlaygraph(fig)
fig.tight_layout()
plt.show()

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('b1 map Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_phi[:,0,0], x=xgo_phi, y=ygo_phi, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_phi[:,0,0], x=xgo_phi, y=ygo_phi, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(6):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
#INTEGRATED S AND R MFPADs in MF

fig, ax = plt.subplots(3,2,sharex=True,sharey=True,figsize=(12, 18))

ax[0,0].contourf(phiM,cosM,MFPAD_R_phi_norm_sum.reshape(200,100).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]])
ax[0,0].set_title("MFPAD R MF")
ax[0,0].set_ylabel('cos(theta) [adm]')

ax[0,1].contourf(phiM,cosM,MFPAD_S_phi_norm_sum.reshape(200,100).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]])
ax[0,1].set_title("MFPAD S MF")

ax[1,0].contourf(phiMM,cosMM,MFPAD_R_phi_LF_sum.T, vmin=0.08, vmax=0.175, levels=20)
ax[1,0].set_title("MFPAD R LF")
ax[1,0].set_ylabel('cos(theta) [adm]')

ax[1,1].contourf(phiMM,cosMM,MFPAD_S_phi_LF_sum.T, vmin=0.08, vmax=0.175, levels=20)
ax[1,1].set_title("MFPAD S LF")

ax[2,0].contourf(phiMM,cosMM,MFPAD_R_phi_LF_norm_sum.T, vmin=4.7, vmax=8.3, levels=20)
ax[2,0].set_title("MFPAD S LF norm")
ax[2,0].set_ylabel('cos(theta) [adm]')
ax[2,0].set_xlabel('phi [DEG]')

ax[2,1].contourf(phiMM,cosMM,MFPAD_S_phi_LF_norm_sum.T, vmin=4.7, vmax=8.3, levels=20)
ax[2,1].set_title("MFPAD S LF norm")
ax[2,1].set_xlabel('phi [DEG]')



plt.show()

In [ ]:
mPECD_phi_sum.shape
mPECD_LF_phi_sum.shape

In [ ]:
#NOTE: has to be transposed

# interpolation: [ 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'.]

fig0=plt.figure()
plt.contourf(phiM,cosM,mPECD_phi_sum.T, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], cmap=cmap_temp)
# plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='none', cmap=cmap_temp, aspect="auto")
plt.title("mPECD contour theory")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

fig1=plt.figure()
plt.imshow(mPECD_LF_phi_sum, vmin=-0.22, vmax=0.22, origin="lower", extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
plt.title("imshow rotated quadratic")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

fig2=plt.figure()
plt.contourf(phiM,cosM, mPECD_LF_phi_sum.T, vmin=-0.2, vmax=0.2, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], levels= 40, cmap=cmap_temp)
plt.title("contourf rotated")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

# fig3=plt.figure()
# plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, origin="lower", extent=[phiM[0],phiM[-1],cosM[-1],cosM[0]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
# plt.title("imshow theory quadratic interpolation")
# plt.xlabel('phi [DEG]')
# plt.ylabel('cos(theta) [adm]')
# plt.colorbar()

